In [2]:
# # from coco_custom import COCO_loader

# categories = ["bus", "cat", "pizza"]

# train_cl = COCO_loader()
# for category in categories:
#     train_cl.save_images_to_folder(category)
#     with open(f'./train_manifest_{category}.pkl', 'rb') as handle:
#         data = pickle.load(handle)

#     print(len(data))

# val_cl = COCO_loader(dataType='val')
# for category in categories:
#     val_cl.save_images_to_folder(category)
#     with open(f'./val_manifest_{category}.pkl', 'rb') as handle:
#         data = pickle.load(handle)

#     print(len(data))

loading annotations into memory...
Done (t=13.22s)
creating index...
index created!


100%|██████████| 2791/2791 [01:18<00:00, 35.52it/s]


1046


100%|██████████| 2818/2818 [01:20<00:00, 34.91it/s] 


1150


...

562


100%|██████████| 1480/1480 [00:07<00:00, 208.84it/s]


655


100%|██████████| 1117/1117 [00:10<00:00, 105.95it/s]

464


In [21]:
from dataset import MyDataset
from torch.utils.data import DataLoader

import cv2
import numpy as np
import pickle

import os

categories = ["bus", "cat", "pizza"]

train_dataset = MyDataset(categories=categories, split="train")
val_dataset = MyDataset(categories=categories, split="val")


num_images = 3
for data_split in ["train", "val"]:
    all_images = []
    for category in categories:

        split_path = f"./{data_split}_manifest_{category}.pkl"
        with open(split_path, "rb") as handle:
            data = pickle.load(handle)
        # print(category, data_split)
        # path = os.path.join(data_path, category, data_split)
        # path = os.path.join(path, category)
        # image_list = os.listdir(path)[:num_images]
        # print(image_list)
        cat_images = []
        for d in data[:num_images]:
            x, y, w, h = d["bboxes"][0]
            img = cv2.imread(d["file_name"])
            img = cv2.rectangle(
                img, (x, y), (x + w, y + h), color=(36, 255, 0), thickness=2
            )
            img = cv2.putText(
                img,
                org=(x, y - 10),
                text=d["category"],
                color=(255, 0, 0),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.8,
                thickness=2,
            )
            cat_images.append(img)
        tot_images = np.concatenate(cat_images, axis=1)

        all_images.append(tot_images)

    all_images = np.concatenate(all_images, axis=0)

    cv2.imwrite(f"./{data_split}_sampled_images.png", all_images)


In [28]:
import torch
import torch.nn as nn


class MySkipBlock(nn.Module):
            def __init__(self, in_ch, out_ch, downsample=False, skip_connections=True):
                super(MySkipBlock, self).__init__()

                # since it only works for out_ch are same or double as in_ch 
                assert out_ch == in_ch or out_ch == 2*in_ch

                self.downsample = downsample
                self.skip_connections = skip_connections
                self.in_ch = in_ch
                self.out_ch = out_ch

                self.convo1 = nn.Conv2d(in_ch, out_ch, 3, stride=1, padding=1)
                self.convo2 = nn.Conv2d(out_ch, out_ch, 3, stride=1, padding=1)
                norm_layer1 = nn.BatchNorm2d
                norm_layer2 = nn.BatchNorm2d
                self.bn1 = norm_layer1(out_ch)
                self.bn2 = norm_layer2(out_ch)
                if downsample:
                    self.downsampler = nn.Conv2d(in_ch, out_ch, 1, stride=2)

            def forward(self, x):
                identity = x                                     
                out = self.convo1(x)                              
                out = self.bn1(out)                              
                out = torch.nn.functional.relu(out)

                out = self.convo2(out)                              
                out = self.bn2(out)                              
                out = torch.nn.functional.relu(out)

                if self.downsample:
                    out = self.downsampler(out)
                    identity = self.downsampler(identity)
                
                if self.skip_connections:
                    if self.in_ch == self.out_ch:
                        out += identity                              
                    else:
                        # out[:,:self.in_ch,:,:] += identity
                        # out[:,self.in_ch:,:,:] += identity
                        torch.cat((out[:, :self.in_ch, :, :] + identity, out[:, self.in_ch:, :, :] + identity), dim=1)
                return out
            

class MyNet(nn.Module):
    """
    The acronym 'LOAD' stands for 'LOcalization And Detection'.
    LOADnet2 uses both convo and linear layers for regression

    Class Path:   DLStudio  ->  DetectAndLocalize  ->  LOADnet2
    """ 
    def __init__(self, skip_connections=True, depth=8):
        super(MyNet, self).__init__()

        self.depth = depth // 2
        self.conv = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1  = nn.BatchNorm2d(64)
        self.bn2  = nn.BatchNorm2d(128)
        self.skip64_arr = nn.ModuleList()
        for i in range(self.depth):
            self.skip64_arr.append(MySkipBlock(64, 64,
                                                    skip_connections=skip_connections))
        self.skip64ds = DLStudio.DetectAndLocalize.SkipBlock(64, 64, 
                                    downsample=True, skip_connections=skip_connections)
        self.skip64to128 = DLStudio.DetectAndLocalize.SkipBlock(64, 128, 
                                                    skip_connections=skip_connections )
        self.skip128_arr = nn.ModuleList()
        for i in range(self.depth):
            self.skip128_arr.append(DLStudio.DetectAndLocalize.SkipBlock(128, 128,
                                                    skip_connections=skip_connections))
        self.skip128ds = DLStudio.DetectAndLocalize.SkipBlock(128,128,
                                    downsample=True, skip_connections=skip_connections)
        self.fc1 =  nn.Linear(2048, 1000)
        self.fc2 =  nn.Linear(1000, 5)

        ##  for regression
        self.conv_seqn = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.fc_seqn = nn.Sequential(
            nn.Linear(16384, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 4)        ## output for the 4 coords (x_min,y_min,x_max,y_max) of BBox
        )

    def forward(self, x):
        x = nn.MaxPool2d(2,2)(torch.nn.functional.relu(self.conv(x)))          
        ## The labeling section:
        x1 = x.clone()
        for i,skip64 in enumerate(self.skip64_arr[:self.depth//4]):
            x1 = skip64(x1)                
        x1 = self.skip64ds(x1)
        for i,skip64 in enumerate(self.skip64_arr[self.depth//4:]):
            x1 = skip64(x1)                
        x1 = self.bn1(x1)
        x1 = self.skip64to128(x1)
        for i,skip128 in enumerate(self.skip128_arr[:self.depth//4]):
            x1 = skip128(x1)                
        x1 = self.bn2(x1)
        x1 = self.skip128ds(x1)
        for i,skip128 in enumerate(self.skip128_arr[self.depth//4:]):
            x1 = skip128(x1)                
        x1 = x1.view(-1, 2048 )
        x1 = torch.nn.functional.relu(self.fc1(x1))
        x1 = self.fc2(x1)
        ## The Bounding Box regression:
        x2 = self.conv_seqn(x)
        # flatten
        x2 = x2.view(x.size(0), -1)
        x2 = self.fc_seqn(x2)
        return x1,x2


tensor([0, 2, 2, 0])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

In [41]:
import torch
import torch.nn as nn
from torchsummary import summary

from tqdm import tqdm

class ResnetBlock(nn.Module):
    def __init__(self, in_channels, 
                 out_channels, 
                 stride = 1, 
                 downsample = None):
        
        '''
        in_channels: input channels, 
        out_channels: output channels, 
        downsample: set to None but modified based on stride and out channels
        '''
        super(ResnetBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.downsample = None

        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, 
                                  kernel_size = 3, 
                                  stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        if downsample:
            self.downsample = nn.Conv2d(in_channels, out_channels, 1, stride=2)
        

        self.relu = nn.ReLU()

        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        else:
            if self.in_channels != self.out_channels:
                out = torch.cat((out[:, :self.in_channels, :, :] + residual, 
                        out[:, self.in_channels:, :, :] + residual), dim=1)
                return self.relu(out)
        
        out += residual
        out = self.relu(out)
        return out
    
class HW5Net(nn.Module):
    def __init__(self, in_ch, 
                #  out_ch, 
                 ngf=8, 
                 n_downsample=2, 
                 n_blocks=4) -> None:
        super(HW5Net, self).__init__()
        model_layers = [nn.ReflectionPad2d(3) ,
                    nn.Conv2d(in_ch, 
                              ngf, 
                              kernel_size =7, 
                              padding=0),
                    nn.BatchNorm2d(ngf) ,
                    nn.ReLU(True)]
        
        for i in range(n_downsample):
            mult = 2**i
            model_layers += [nn.Conv2d(ngf*mult, ngf*mult*2, kernel_size=3, 
                                padding=1, stride=2), 
                        nn.BatchNorm2d(ngf*mult*2),
                        nn.ReLU(True)]
        
        mult = 2**n_downsample
        # for i in range(n_blocks):
        model_layers += [ResnetBlock(ngf*mult, ngf*mult*2, stride=1, downsample=False)]
        mult *= 2
        for _ in range(n_blocks - 1):
            model_layers += [ResnetBlock(ngf*mult, ngf*mult*2, stride=2, downsample=True)]
            mult *=2

        self.model = nn.Sequential(*model_layers)

        classification_head = [nn.Linear(2048*2*2, 1024),
                                nn.ReLU(inplace=True),
                                nn.Linear(1024, 3)]
        self.class_head = nn.Sequential(*classification_head)
        

        #  for regression
        self.bbox_head_conv = nn.Sequential(*[
            nn.Conv2d(in_channels=2048, out_channels=1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)])
        
        self.bbox_head_seq =  nn.Sequential(*[
            nn.Linear(512*2*2, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 4)        ## output for the 4 coords (x_min,y_min,x_max,y_max) of BBox
        ])
        
        # self.bbox_head = nn.Sequential(*bbox_head)


    def forward(self, x):
        ft = self.model(x)
        # print(ft.shape)
        class_out = self.class_head(ft.view(1, -1))
        # print(class_out)

        regression_out = self.bbox_head_conv(ft)
        # print(regression_out.shape)
        regression_out = self.bbox_head_seq(regression_out.view(1, -1))
        # print(regression_out)

        return class_out, regression_out

# model = HW5Net(3, 5)
# sample = torch.rand((3, 256, 256))

# model(sample.unsqueeze(0))
# sample.shape

# import torchinfo

# torchinfo.summary(model, (3, 256, 256), batch_dim=0)



(tensor([[0.1734, 0.1523, 0.0170]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.0287, -0.0136,  0.0113,  0.0064]], grad_fn=<AddmmBackward0>))

In [26]:
def train(
    net, train_dataloader, epochs=10, net_name="net_name"
) -> None:
    device = "cuda" if torch.cuda.is_available() == True else "cpu"
    net.train()
    net = net.to(device=device)

    criterion_CE = nn.CrossEntropyLoss()
    criterion_MSE = nn.MSELoss()

    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.99))

    # tot_loss = []
    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        loss_flag = 1e32
        for i, data in enumerate(train_dataloader):
            inputs, labels, bbox = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            bbox = bbox.to(device)
            optimizer.zero_grad()
            output_labels, output_bboxes = net(inputs)
            loss_CE = criterion_CE(output_labels, labels)
            loss_CE.backward(retain_graph=True)
            optimizer.step()
            running_loss += loss.item()
            if (i + 1) % 100 == 0:
                # opening the csv file in 'w+' mode
                # print("[epoch]: %d, batch: %5d] loss: %.3f" %(epoch+1, i+1, running_loss / 100))
                data = [epoch + 1, i + 1, running_loss / 100]
                file = open("results/" + net_name + ".csv", "a", newline="")
                # writing the data into the file
                with file:
                    write = csv.writer(file)
                    write.writerow(data)

                # tot_loss.append(running_loss/100)
                if running_loss < loss_flag:
                    loss_flag = running_loss
                    torch.save(net.state_dict(), "results/" + net_name + ".pt")
                running_loss = 0.0


512